## Select Individual State for Mapping

In [1]:
import pathlib
import pandas as pd
import numpy as np
import geopandas as gp
import shapely.geometry as geom
import folium

#### Set input file location and read into Pandas

In [2]:
input_file = 'od_stats.csv.gz'
ODpath = pathlib.Path('../data/OD/')
ODfile = ODpath.joinpath(input_file)

with ODfile.open(mode='r') as fid:
#    df_All = pd.read_csv(ODfile, dtype = {'w_geocode': object, 'h_geocode': object})
    df_All = pd.read_csv(ODfile, dtype = {'w_geocode': object})

print ('input file: ',ODfile)
print('\nrecords loaded to dataframe:', "{:,}".format(len(df_All)),'\n\n')
df_All.head()

input file:  ../data/OD/od_stats.csv.gz

records loaded to dataframe: 17,233 




,w_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon
0,010010205001001,1100,536,426,138,534,412,154,13,540,547,32.456740,-86.415025
1,010030107032109,1009,498,406,105,517,332,160,0,584,425,30.667644,-87.849564
2,010030112023027,1088,173,656,259,93,413,582,0,0,1088,30.518815,-87.888250
3,010030115021041,1280,560,522,198,719,397,164,0,805,475,30.372959,-87.684560
4,010059505002038,1448,432,793,223,105,1020,323,1448,0,0,31.801006,-85.332896


#### Set the state to analyze

In [3]:
state_id = '21'

#### Set the output file location

In [4]:
output_loc = pathlib.Path('../data/')
output_file = output_loc.joinpath(state_id+'.geojson')
print ('path for output:',output_file)

path for output: ../data/21.geojson


#### Drop all records where the work location does not being with the chosen state_id

In [5]:
df = df_All.drop(df_All[~df_All['w_geocode'].str.startswith(state_id)].index)
df = df.reset_index(drop=True)
print('\nnew dataframe length: ', "{:,}".format(len(df)),'\n\n')
df.head(1)


new dataframe length:  223 




,w_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon
0,210099505002000,1238,137,854,247,71,599,568,0,0,1238,37.010798,-85.906837


#### Find the unique block codes with 1000+ employees

In [6]:
df_unique_list = df.w_geocode.unique().tolist()
print ('number of unique cenus blocks with over 1000 employees in state FIPS code: ',state_id, 'is:', len(df_unique_list))

number of unique cenus blocks with over 1000 employees in state FIPS code:  21 is: 223


#### Add a county code column

In [7]:
df['county'] = df.w_geocode.str[3:5]
df.head()

,w_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,county
0,210099505002000,1238,137,854,247,71,599,568,0,0,1238,37.010798,-85.906837,09
1,210150702005017,1327,260,795,272,164,637,526,208,590,529,38.982623,-84.625940,15
2,210150703011003,1455,142,919,394,45,367,1043,1177,45,233,38.977810,-84.625726,15
3,210150703011004,1832,297,1100,435,150,835,847,1341,276,215,38.973936,-84.634155,15
4,210150703011019,1557,283,1014,260,57,447,1053,1394,95,68,38.970680,-84.622564,15


#### The unique counties containing the blocks with 1000+ employees

In [8]:
df.county.unique()

array(['09', '15', '19', '21', '29', '35', '37', '41', '47', '49', '51',
       '53', '59', '67', '69', '71', '73', '81', '83', '89', '93', '95',
       '99', '01', '07', '11', '13', '17', '25', '45', '55', '77', '79',
       '85', '05', '27'], dtype=object)

In [10]:
df_test = df.copy() 
df_test.drop(['county'], axis=1, inplace=True)
df_test.head()

,w_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon
0,210099505002000,1238,137,854,247,71,599,568,0,0,1238,37.010798,-85.906837
1,210150702005017,1327,260,795,272,164,637,526,208,590,529,38.982623,-84.625940
2,210150703011003,1455,142,919,394,45,367,1043,1177,45,233,38.977810,-84.625726
3,210150703011004,1832,297,1100,435,150,835,847,1341,276,215,38.973936,-84.634155
4,210150703011019,1557,283,1014,260,57,447,1053,1394,95,68,38.970680,-84.622564


In [11]:
df_test.w_lat = df_test.w_lat.astype('object')
df_test.w_lon = df_test.w_lon.astype('object')
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 13 columns):
w_geocode    223 non-null object
S000         223 non-null int64
SA01         223 non-null int64
SA02         223 non-null int64
SA03         223 non-null int64
SE01         223 non-null int64
SE02         223 non-null int64
SE03         223 non-null int64
SI01         223 non-null int64
SI02         223 non-null int64
SI03         223 non-null int64
w_lat        223 non-null object
w_lon        223 non-null object
dtypes: int64(10), object(3)
memory usage: 22.7+ KB


In [12]:
df_group = df_test.groupby(['w_geocode','w_lat','w_lon']).sum()
df_group.head(10)

,,,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03
w_geocode,w_lat,w_lon,,,,,,,,,,
210099505002000,37.010798,-85.906837,1238,137,854,247,71,599,568,0,0,1238
210150702005017,38.982623,-84.625940,1327,260,795,272,164,637,526,208,590,529
210150703011003,38.977810,-84.625726,1455,142,919,394,45,367,1043,1177,45,233
210150703011004,38.973936,-84.634155,1832,297,1100,435,150,835,847,1341,276,215
210150703011019,38.970680,-84.622564,1557,283,1014,260,57,447,1053,1394,95,68
210150703011029,38.945641,-84.613903,1508,290,952,266,75,625,808,370,1102,36
210150703111008,39.019192,-84.639989,3103,806,1676,621,626,904,1573,1513,1078,512
210150703111044,39.017994,-84.627632,3715,1071,2013,631,267,1582,1866,217,24,3474
210150703112018,39.053239,-84.634528,1048,133,654,261,56,281,711,5,84,959


#### Create a dataframe of unique blocks contain centroid

In [13]:
df_unique_blocks = df.drop_duplicates(['w_geocode'], keep='last')
df_unique_blocks.drop(['county'], axis=1, inplace=True)
df_blocks = df_unique_blocks.copy().reset_index(drop=True)
print('\nnumber of unique blocks',len(df_blocks),'\n')
df_blocks.head()


number of unique blocks 223 



,w_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon
0,210099505002000,1238,137,854,247,71,599,568,0,0,1238,37.010798,-85.906837
1,210150702005017,1327,260,795,272,164,637,526,208,590,529,38.982623,-84.625940
2,210150703011003,1455,142,919,394,45,367,1043,1177,45,233,38.977810,-84.625726
3,210150703011004,1832,297,1100,435,150,835,847,1341,276,215,38.973936,-84.634155
4,210150703011019,1557,283,1014,260,57,447,1053,1394,95,68,38.970680,-84.622564


#### Create a column for the dominant industry sector of the block

In [20]:
#df_blocks[['SI01','SI02','SI03']].max(axis=1)
df_blocks['industry_max']=df_blocks[['SI01','SI02','SI03']].idxmax(axis=1)
df_blocks.head(1)

,w_geocode,S000,SA01,SA02,SA03,SE01,SE02,SE03,SI01,SI02,SI03,w_lat,w_lon,industry_max
0,210099505002000,1238,137,854,247,71,599,568,0,0,1238,37.010798,-85.906837,SI03


In [51]:
def color(industry):
    if industry == 'SI01': 
        col='green'
    elif industry == 'SI02':
        col='red'
    else:
        col='blue'
    return col

#### Create lat lon list for mapping locations with 1000+ employees

In [40]:
locations = df_blocks[['w_lat', 'w_lon']]
locationlist = locations.values.tolist()
len(locationlist)
locationlist[7]

[39.0179944, -84.6276316]

In [94]:
m = folium.Map(location=[37.645556, -84.769722], tiles='cartodbpositron',
                zoom_start=8, control_scale=True, prefer_canvas=True)

legend_html =   '''
                <div style="position: fixed; 
                            bottom: 50px; left: 5px; width: 230px; height: 130px; 
                            border:2px solid grey; z-index:9999; font-size:14px;
                            ">&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>Industry Type </b><br>
                            &nbsp;&nbsp;<i class="fa fa-map-marker fa-2x" style="color:green"></i>&nbsp; Goods Producing &nbsp;<br>
                            &nbsp;&nbsp;<i class="fa fa-map-marker fa-2x" style="color:red"></i>&nbsp; Trade, Transportation, Utilities &nbsp;<br>
                            &nbsp;&nbsp;<i class="fa fa-map-marker fa-2x" style="color:blue"></i>&nbsp; Service &nbsp; 
                </div>
                ''' 

title_html =   '''
                <div style="position: fixed; 
                            top: 10px; left: 450px; width: 500px; height: 30px; 
                            border:0px solid grey; z-index:9999; font-size:24px;
                            "><b>Block locations with 1000+ employees </b><br>
                </div>
                ''' 
    
m.get_root().html.add_child(folium.Element(title_html))    

for point in range(len(locationlist)):
    #folium.Marker(locations[point], popup=df_blocks['w_geocode'][point]).add_to(m)
    folium.Marker(locationlist[point], 
                  popup=df_blocks['w_geocode'][point], 
                  icon= folium.Icon(color=color(df_blocks['industry_max'][point]))).add_to(m)
    
m.get_root().html.add_child(folium.Element(legend_html))    
m

In [ ]:
%time df['geom'] = df.apply(lambda x: ([(x['w_lat'], x['w_lon']),(x['h_lat'],x['h_lon'])]), axis = 1)
df.head()

#### Are there any nulls?

In [ ]:
df_null = df[df.isnull().any(axis=1)]
df_null.head()

#### Remove rows with any nulls

In [ ]:
df = df.dropna(how='any')
df_null = df[df.isnull().any(axis=1)]
df_null.head()

In [ ]:
df.index

In [ ]:
df_unique = df[df['w_geocode'].isin(unique_locations_list)]

In [ ]:
%time df['geometry'] =df.apply(lambda x: geom.LineString([(x['w_lon'], x['w_lat'] ),(x['h_lon'],x['h_lat'])]),axis = 1)


In [ ]:
gdf_ky = gp.GeoDataFrame(df, geometry='geometry')

In [ ]:
gdf_ky.head()

In [ ]:
gdf_ky.crs = {'init' :'epsg:4326'}

In [ ]:
gdf_ky.drop(columns=['S000','SA01','SA02','SA03','SE01','SE02','SE03','SI01','SI02','SI03','w_lat','w_lon','h_lat','h_lon','geom'],axis=1, inplace=True)


In [ ]:
gdf_ky.to_file(output_file, driver="GeoJSON")

In [ ]:
gjson = gdf_ky.geometry.to_json()

In [ ]:
gjson[1:1190]

In [ ]:
df_lex = df_All.drop(df_All[~df_All['w_geocode'].str.startswith('21067')].index)
df_lex = df_lex.reset_index(drop=True)
print(len(df_lex))
df_lex.head()

In [ ]:
m = folium.Map(location=[37.645556, -84.769722], tiles='cartodbpositron',
                zoom_start=7, control_scale=True, prefer_canvas=True)

In [ ]:
#mylocation = [(37.99491, -85.66809559999999), (38.0899342, -84.884275)]
mylocation = df.geom.values[2000]
mylocation = lines[1000]
my_PolyLine=folium.PolyLine(mylocation, line_color='#FF0000',weight=1).add_to(m)
m
print(mylocation)

In [ ]:
%time df_lex['geometry'] = df_lex.apply(lambda x: geom.LineString([(x['w_lon'], x['w_lat'] ), (x['h_lon'],x['h_lat'])]), axis = 1)


In [ ]:
gdf_lex = gp.GeoDataFrame(df_lex, geometry='geometry')

In [ ]:
gdf_lex.drop(columns=['S000','SA01','SA02','SA03','SE01','SE02','SE03','SI01','SI02','SI03','w_lat','w_lon','h_lat','h_lon'],axis=1, inplace=True)


In [ ]:
gdf_lex.crs = {'init' :'epsg:4326'}

In [ ]:
gdf_lex.head()

In [ ]:
gjson = gdf_lex.geometry.to_json()

#### Write LEX data to geojson

In [ ]:
output_lex = output_loc.joinpath('lex_all.geojson')
print (output_lex)

In [ ]:
%time gdf_lex.to_file(output_lex, driver="GeoJSON")

In [ ]:
gdf_lex.head()

In [ ]:
%time gdf_lex_limited = gdf_lex[(gdf_lex['distance'] < 120000)]

In [ ]:
print (len(gdf_lex_limited))
gdf_lex_limited.head()

In [ ]:
output_loc = pathlib.Path('../data/')
output_lex_limited = output_loc.joinpath('lex_limited_output.geojson')
print (output_lex_limited)

In [ ]:
%time gdf_lex_limited.to_file(output_lex_limited, driver="GeoJSON")

#### Convert distance from float with decimal to no decimal string